In [763]:
import simpy
import random
import math
import numpy

In [764]:
RANDOM_SEED = 9782


INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60


CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [765]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Duration of waiting time of the customer between automatic operator and op1 or op2
queue_waiting = []



# The number of customer in the answering system
queue_w_customer = []

# The number of customer waiting operator1
queue_w_operator1 = []

# The number of customer waiting operator2
queue_w_operator2 = []

#Average number of people waiting to be served by operator1.
queue_w_op1 = []

#Average number of people waiting to be served by operator2.
queue_w_op2 = []

#Average number of customers leaving the system unsatisfied either due to
#incorrect routing or due to long waiting times.
unsatisfied_customer=[]

#Utilization of the answering system.
answering_util=[]


# Simulation end time
end_time = 0;



In [766]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        self.queue_t=0
    def call(self):
        global end_time 
        
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        prob=random.uniform(0,10)
        error=random.uniform(0,10)
     
        #The answering system can serve 100 callers simultaneously 
        if len(queue_w_customer)<101:
            yield env.timeout(random.expovariate(CALL_RATE))
            answering_util.append(self.env.now-self.arrival_t)
            queue_w_customer.pop()
            self.queue_t=self.env.now
            
            #The voice recognition system is not perfect and makes a mistake with a 0.1 probability
            if error>1 :
                #operator one with probability 0.3 or to operator 2 with probability 0.7.
                if prob<3:
                    queue_w_operator1.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call1())
                if prob>=3:  
                    queue_w_operator2.append(self.env.now - self.arrival_t)
                    yield self.env.process(self.call2())        
            else:
                #An arriving customer that has been waiting on hold for 10 minutes hangs up and leaves the system (reneging).
                unsatisfied_customer.append(1)
                end_time = self.env.now
                print('%s A caller hangs up -error- at %g' % (self.name, self.env.now))
        else:
            
            print('%s A caller drops without answering.-100 parallel channels- at %g' % (self.name, self.env.now))
            queue_w_customer.pop()


        #arrival to operator1
    def call1(self):
        
        with operator1.request() as req:
            yield req
            if((self.env.now - self.queue_t))<10:
                queue_waiting.append(self.env.now - self.queue_t)
                queue_w_op1.append(1)
                print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
                
                yield self.env.process(self.get_help1_operator1())
                print('%s is done operator1 at %g' % (self.name, self.env.now))
                end_time = self.env.now
            else:
                #unsatisfied due to long waiting times.
                unsatisfied_customer.append(1)
                queue_waiting.append(10)
                queue_w_op1.append(1)
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                
                end_time = self.queue_t+10
            queue_w_operator1.pop()

    #arrival to operator2
    def call2(self):
        
        with operator2.request() as req:
            yield req

            if((self.env.now - self.queue_t))<10:
                queue_waiting.append(self.env.now - self.queue_t)
                queue_w_op2.append(1)
                print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
                yield self.env.process(self.get_help2_operator2())
                print('%s is done operator2 at %g' % (self.name, self.env.now))
                end_time = self.env.now
            else:
                 #unsatisfied due to long waiting times.
                unsatisfied_customer.append(1)
                queue_waiting.append(10)
                queue_w_op2.append(1)
                print('%s A caller hangs up-10 min- at %g' % (self.name, (self.queue_t+10)))
                end_time = self.queue_t+10
            queue_w_operator2.pop()

      
            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [767]:
# Generate new customer that call to call center.
def customer_generator(env, operator1, operator2):
   
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [768]:

#   Generate new break for operator1.
def break_generator1(env, operator1):

    BREAK_TIME_COUNTER1 = 0
    
    while True:
      
 
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        # waits until completing all the customers already waiting for her/him.
        while True:
            with operator1.request() as req:
                yield req
                
                if queue_w_operator1:
                    continue
                else:
                    print('Operator1 gives break at %g' % (env.now))
                    yield env.timeout(BREAK_TIME)
                    print('Operator1 break done at %g' % (env.now))
                    BREAK_TIME_COUNTER1+=1
                  
                    break

      

In [769]:
def break_generator2(env, operator2):
    """Generate new break for operator2."""
    
    BREAK_TIME_COUNTER2 = 0
    
    while True:
 
           
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        
        # waits until completing all the customers already waiting for her/him.
        while True:
            with operator2.request() as req:
                yield req
                if queue_w_operator2:
                    continue
                else:
                    print('Operator2 gives break at %g' % (env.now))
                    yield env.timeout(BREAK_TIME)
                    print('Operator2 break done at %g' % (env.now))
                    BREAK_TIME_COUNTER2+=1
                    
                    break

In [770]:
#variables of statistics

#sum of 10 1000 customer data utilization of operator1 
sumUtil1KOP1=0
#sum of 10 1000 customer data utilization of operator2 
sumUtil1KOP2=0

#sum of waiting times of 1k customer
sumOf1kWaitingTimes=0

#Maximum Total Waiting Time to Total System Time Ratio,
maxRatio1K=0

#Sum of operater1 waiting times
sumof1kOp1wTimes=0

#Sum of operater2 waiting times
sumof1kOp2wTimes=0

#Sum of Unsatisfied costumer leaving times
sumof1kUnwTimes=0

#Sum of Util time answering system
sumof1kUtilAnswering=0



#sum of 10 5000 customer data utilization of operator1 
sumUtil5KOP1=0
#sum of 10 5000 customer data utilization of operator2 
sumUtil5KOP2=0

#sum of waiting times of 5k customer
sumOf5kWaitingTimes=0

#Maximum Total Waiting Time to Total System Time Ratio,
maxRatio5K=0

#Sum of operater1 waiting times
sumof5kOp1wTimes=0

#Sum of operater2 waiting times
sumof5kOp2wTimes=0

#Sum of Unsatisfied costumer leaving times
sumof5kUnwTimes=0

#Sum of Util time answering system
sumof5kUtilAnswering=0



In [771]:
#1k
for i in range(10):
    RANDOM_SEED+=1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1)
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(customer_generator(env, operator1, operator2))
    env.process(break_generator1(env, operator1))
    env.process(break_generator2(env, operator2))
    env.run(until=7000)
    
    #statistic calculations
    
    
    print(end_time)
    
    #utilization of operators
    serviceTimeOfOP1=sum(service_times_operator1)
    utilizaztionOfOP1=serviceTimeOfOP1/end_time
    sumUtil1KOP1+=utilizaztionOfOP1
    serviceTimeOfOP2=sum(service_times_operator2)
    utilizaztionOfOP2=serviceTimeOfOP2/end_time
    sumUtil1KOP2+=utilizaztionOfOP2
    
    #total waiting time
    sumOf1kWaitingTimes+=sum(queue_waiting)
    
    #Sum of waiting time of operator1
    sumof1kOp1wTimes+=(sum(queue_w_op1))
    
    #Sum of waiting time of operator2
    sumof1kOp2wTimes+=(sum(queue_w_op2))
    
    #Sum of Unsatisfied costumer leaving times
    sumof1kUnwTimes+=(sum(unsatisfied_customer))
    
    #Sum of Util time answering system

    sumof1kUtilAnswering+=(sum(answering_util)/end_time)


    
    #Max Ratio
    maxRatio1K=max(sum(queue_waiting)/end_time,maxRatio1K)

    
    #reset variables
    service_times_operator1 = []

    service_times_operator2 = []
    
    queue_waiting = []
    
    queue_w_op1 = []
    
    queue_w_op2 = []

    queue_w_customer = []

    queue_w_operator1 = []

    queue_w_operator2 = []
    
    end_time = 0
    
    unsatisfied_customer= []
    
    answering_util=[]
    
    
    
    

Customer 1 initiated a call at 3.01324
Customer 2 initiated a call at 3.5674
Customer 3 initiated a call at 3.92895
Customer 4 initiated a call at 3.98808
Customer 2 is assigned to a operator2 at 4.98243
Customer 3 A caller hangs up -error- at 7.64086
Customer 5 initiated a call at 7.71364
Customer 2 is done operator2 at 9.63242
Customer 1 is assigned to a operator2 at 9.63242
Customer 1 is done operator2 at 14.4318
Customer 4 is assigned to a operator2 at 14.4318
Customer 4 is done operator2 at 21.3165
Customer 5 is assigned to a operator2 at 21.3165
Customer 5 is done operator2 at 24.6464
Customer 6 initiated a call at 24.9806
Customer 7 initiated a call at 25.2417
Customer 8 initiated a call at 29.1295
Customer 7 is assigned to a operator2 at 29.5136
Customer 7 is done operator2 at 33.9539
Customer 6 is assigned to a operator2 at 33.9539
Operator1 gives break at 34.3972
Customer 8 A caller hangs up -error- at 36.8438
Customer 9 initiated a call at 37.1984
Operator1 break done at 37.

Customer 469 initiated a call at 2642.37
Operator2 break done at 2644.95
Customer 469 is assigned to a operator2 at 2644.95
Customer 470 initiated a call at 2645.71
Customer 471 initiated a call at 2648.44
Customer 469 is done operator2 at 2650.22
Customer 467 is assigned to a operator2 at 2650.22
Customer 467 is done operator2 at 2651.33
Customer 470 is assigned to a operator2 at 2651.33
Customer 472 initiated a call at 2651.55
Customer 470 is done operator2 at 2655.83
Customer 471 is assigned to a operator2 at 2655.83
Customer 472 A caller hangs up -error- at 2658.83
Customer 471 is done operator2 at 2660.6
Operator2 gives break at 2660.6
Customer 473 initiated a call at 2662.04
Operator2 break done at 2663.6
Customer 473 is assigned to a operator2 at 2668.38
Customer 474 initiated a call at 2668.5
Customer 456 is done operator1 at 2669.95
Customer 459 A caller hangs up-10 min- at 2608.7
Customer 461 A caller hangs up-10 min- at 2618.69
Customer 468 A caller hangs up-10 min- at 2662.

Customer 940 A caller hangs up-10 min- at 5503.21
Customer 942 A caller hangs up-10 min- at 5504.25
Customer 936 A caller hangs up-10 min- at 5507.15
Customer 943 A caller hangs up-10 min- at 5513.04
Customer 941 A caller hangs up-10 min- at 5513.5
Customer 945 A caller hangs up-10 min- at 5514.81
Customer 947 is assigned to a operator1 at 5535.78
Customer 953 initiated a call at 5537.61
Customer 949 is assigned to a operator2 at 5541.38
Customer 954 initiated a call at 5542.06
Customer 947 is done operator1 at 5544.79
Customer 953 is assigned to a operator1 at 5544.79
Customer 949 is done operator2 at 5545.84
Customer 952 is assigned to a operator2 at 5545.84
Customer 952 is done operator2 at 5547.09
Customer 954 is assigned to a operator2 at 5547.09
Customer 955 initiated a call at 5551.75
Customer 954 is done operator2 at 5551.77
Customer 955 is assigned to a operator2 at 5552.93
Customer 955 is done operator2 at 5557.36
Customer 956 initiated a call at 5558.6
Customer 956 is assign

Customer 415 is assigned to a operator1 at 2477.08
Customer 416 is done operator2 at 2478.34
Customer 417 is assigned to a operator2 at 2478.34
Customer 415 is done operator1 at 2483.06
Customer 417 is done operator2 at 2483.32
Customer 420 is assigned to a operator2 at 2483.32
Customer 421 initiated a call at 2483.68
Customer 422 initiated a call at 2484.25
Customer 420 is done operator2 at 2486.6
Customer 419 is assigned to a operator2 at 2486.6
Customer 419 is done operator2 at 2487.46
Operator1 gives break at 2488.08
Operator1 break done at 2491.08
Customer 422 is assigned to a operator1 at 2493.07
Customer 421 is assigned to a operator2 at 2494.12
Customer 422 is done operator1 at 2496.03
Customer 423 initiated a call at 2498.47
Customer 421 is done operator2 at 2499.04
Customer 424 initiated a call at 2499.55
Customer 423 is assigned to a operator1 at 2502.56
Customer 425 initiated a call at 2504.45
Customer 426 initiated a call at 2508.24
Customer 424 is assigned to a operator2 

KeyboardInterrupt: 

In [ ]:
#1k

#utilization average of operator 1
utilOf1KOP1=sumUtil1KOP1/10

#utilization average of operator 2
utilOf1KOP2=sumUtil1KOP2/10

#average of waiting times 
avg1kWaitingTimes=sumOf1kWaitingTimes/10

#average of operator1 waiting times
avgof1kOp1wTimes=sumof1kOp1wTimes/10

#average of operator2 waiting times
avgof1kOp2wTimes=sumof1kOp2wTimes/10

#average of Unsatisfied Customer leaving times
avgof1kUnwTimes=sumof1kUnwTimes/10

#average of Util time answering system

avgof1kUtilAnswering=sumof1kUtilAnswering/(10*100)




print('1K AVERAGE STATISTICS')

print('Utilization of The answering System %g' % avgof1kUtilAnswering)
print('Utilization of operator1 %g' % utilOf1KOP1)
print('Utilization of operator1 %g' % utilOf1KOP1)
print('Utilization of operator2 %g' % utilOf1KOP2)
print('Average waiting times of customers %g' % avg1kWaitingTimes)
print('Maximum Total Waiting Time to Total System Time Ratio %g' % maxRatio1K )
print('Average number of people waiting to be served by operator1. %g' % avgof1kOp1wTimes )
print('Average number of people waiting to be served by operator2. %g' % avgof1kOp2wTimes )
print('Average number of Unsatisfied Customer leaving. %g' % avgof1kUnwTimes )





In [ ]:
CUSTOMER_COUNT=5000
for i in range(10):
    RANDOM_SEED+=1
    env = simpy.Environment()
    operator1 = simpy.Resource(env, capacity = 1)
    operator2 = simpy.Resource(env, capacity = 1)
    env.process(customer_generator(env, operator1, operator2))
    env.process(break_generator1(env, operator1))
    env.process(break_generator2(env, operator2))
    env.run(until=32000)
    
     #statistic calculations
    
    
    
    
    #utilization of operators
    serviceTimeOfOP1=sum(service_times_operator1)
    utilizaztionOfOP1=serviceTimeOfOP1/end_time
    sumUtil5KOP1+=utilizaztionOfOP1
    serviceTimeOfOP2=sum(service_times_operator2)
    utilizaztionOfOP2=serviceTimeOfOP2/end_time
    sumUtil5KOP2+=utilizaztionOfOP2
    
    #total waiting time
    sumOf5kWaitingTimes+=sum(queue_waiting)
    
    #Max Ratio
    maxRatio5K=max(sum(queue_waiting)/end_time,maxRatio5K)
    
    #Sum of waiting time of operator1
    sumof5kOp1wTimes+=sum(queue_w_op1)
    
    #Sum of waiting time of operator2
    sumof5kOp2wTimes+=sum(queue_w_op2)
    
    #Sum of Unsatisfied costumer leaving times
    sumof5kUnwTimes+=(sum(unsatisfied_customer))
    
    #Sum of Util time answering system

    sumof5kUtilAnswering+=(sum(answering_util)/end_time)
    
    
    #reset variables
    service_times_operator1 = []

    service_times_operator2 = []

    queue_waiting = []
    
    queue_w_op1 = []
    
    queue_w_op2 = []

    queue_w_customer = []

    queue_w_operator1 = []

    queue_w_operator2 = []
    
    end_time = 0
    
    unsatisfied_customer=[]
    
    answering_util= []

In [ ]:
#5k

#utilization average of operator 1
utilOf5KOP1=sumUtil5KOP1/10

#utilization average of operator 2
utilOf5KOP2=sumUtil5KOP2/10

#average of waiting times 
avg5kWaitingTimes=sumOf5kWaitingTimes/10

#average of operator1 waiting times
avgof5kOp1wTimes=sumof5kOp1wTimes/10

#average of operator2 waiting times
avgof5kOp2wTimes=sumof5kOp2wTimes/10

#average of Unsatisfied Customer leaving times
avgof5kUnwTimes=sumof5kUnwTimes/10

#average of Util time answering system
avgof5kUtilAnswering=sumof5kUtilAnswering/(10*100)

print('5K AVERAGE STATISTICS')

print('Utilization of the answering System %g' % avgof5kUtilAnswering)
print('Utilization of operator1 %g' % utilOf5KOP1)
print('Utilization of operator2 %g' % utilOf5KOP2)
print('Average waiting times of customers %g' % avg5kWaitingTimes)
print('Maximum Total Waiting Time to Total System Time Ratio %g' % maxRatio5K )
print('Average number of people waiting to be served by operator1. %g' % avgof5kOp1wTimes )
print('Average number of people waiting to be served by operator2. %g' % avgof5kOp2wTimes )
print('Average number of Unsatisfied Customer leaving. %g' % avgof5kUnwTimes )


